<a href="https://colab.research.google.com/github/aadhamashraf/Fraud-Detection-Interpretability-and-Explainability/blob/main/final_models/3%20-%20Isolation%20Forest%20with%20Experimental%20Scenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I'm following the paper *Performance Analysis of Isolation Forest Algorithm in Fraud Detection of Credit Card Transactions.* I have reproduced the four experimental scenarios outlined in the paper using the Isolation Forest algorithm.**

In [ ]:
import shap
import random
import pandas as pd
import numpy as np
import lime.lime_tabular
from itertools import product
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import IsolationForest , RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score , classification_report , ConfusionMatrixDisplay , roc_curve,auc , precision_recall_curve
from lime.lime_tabular import LimeTabularExplainer
from sklearn.inspection import PartialDependenceDisplay , permutation_importance

In [ ]:
df = pd.read_csv("/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv")

In [ ]:
df.head()

In [ ]:
print(df['isFraud'].value_counts(normalize=True))

In [ ]:
df = df.drop(columns=["nameOrig", "nameDest"])

df["type"] = LabelEncoder().fit_transform(df["type"])

y = df["isFraud"]
X = df.drop(columns=["isFraud", "isFlaggedFraud"])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
df.columns

# **4 Scenarios Mentioned inside the Paper**

## **Scenario 1: Train/Test Split Optimization**
* Try split ratios: 90:10, 80:20, 70:30, 60:40
* Use full features, evaluate with AUCPR, ROC-AUC, Precision, Recall, F1

In [ ]:
split_ratios = [(0.9, 0.1), (0.8, 0.2), (0.7, 0.3), (0.6, 0.4)]

results = []

for train_size, test_size in split_ratios:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, stratify=y, random_state=42)

    model = IsolationForest(n_estimators=100, max_samples=128, contamination=0.001, random_state=42)
    model.fit(X_train)

    y_pred = model.predict(X_test)
    y_pred = [1 if x == -1 else 0 for x in y_pred]

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)

    precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred)
    aucpr = auc(recall_vals, precision_vals)

    results.append((train_size, precision, recall, f1, roc, aucpr))

for r in results:
    print(f"Train Size: {int(r[0]*100)}% | Precision: {r[1]:.4f}, Recall: {r[2]:.4f}, F1: {r[3]:.4f}, ROC AUC: {r[4]:.4f}, AUCPR: {r[5]:.4f}")

Train Size: 90% | Precision: 0.0069, Recall: 0.0049, F1: 0.0057, ROC AUC: 0.5020, AUCPR: 0.0065
Train Size: 80% | Precision: 0.0040, Recall: 0.0030, F1: 0.0035, ROC AUC: 0.5010, AUCPR: 0.0042
Train Size: 70% | Precision: 0.0011, Recall: 0.0008, F1: 0.0009, ROC AUC: 0.4999, AUCPR: 0.0016
Train Size: 60% | Precision: 0.0037, Recall: 0.0027, F1: 0.0031, ROC AUC: 0.5009, AUCPR: 0.0038


## **Scenario 2: Feature Selection**

* Use Random Forest to select top N features (1–10)
* Repeat Isolation Forest using top N features from Scenario 1 best split


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_scaled, y)
importances = rf.feature_importances_
feature_ranks = np.argsort(importances)[::-1]
features_sorted = X.columns[feature_ranks]

scenario2_results = []

X_df = pd.DataFrame(X_scaled, columns=X.columns)

for n in range(1, 11):
    top_features = features_sorted[:n]
    X_selected = X_df[top_features]

    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.4, stratify=y, random_state=42)

    model = IsolationForest(n_estimators=100, max_samples=128, contamination=0.001, random_state=42)
    model.fit(X_train)
    y_pred = model.predict(X_test)
    y_pred = [1 if x == -1 else 0 for x in y_pred]

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred)
    aucpr = auc(recall_vals, precision_vals)

    scenario2_results.append((n, precision, recall, f1, roc, aucpr))

for res in scenario2_results:
    print(f"Top {res[0]} features | Precision: {res[1]:.4f}, Recall: {res[2]:.4f}, F1: {res[3]:.4f}, ROC AUC: {res[4]:.4f}, AUCPR: {res[5]:.4f}")

X does not have valid feature names, but IsolationForest was fitted with feature names
X does not have valid feature names, but IsolationForest was fitted with feature names
X does not have valid feature names, but IsolationForest was fitted with feature names
X does not have valid feature names, but IsolationForest was fitted with feature names
X does not have valid feature names, but IsolationForest was fitted with feature names
X does not have valid feature names, but IsolationForest was fitted with feature names
X does not have valid feature names, but IsolationForest was fitted with feature names
X does not have valid feature names, but IsolationForest was fitted with feature names
X does not have valid feature names, but IsolationForest was fitted with feature names
X does not have valid feature names, but IsolationForest was fitted with feature names


Top 1 features | Precision: 0.0009, Recall: 0.0006, F1: 0.0007, ROC AUC: 0.4999, AUCPR: 0.0014
Top 2 features | Precision: 0.0008, Recall: 0.0006, F1: 0.0007, ROC AUC: 0.4998, AUCPR: 0.0013
Top 3 features | Precision: 0.0444, Recall: 0.0335, F1: 0.0382, ROC AUC: 0.5163, AUCPR: 0.0396
Top 4 features | Precision: 0.0851, Recall: 0.0661, F1: 0.0744, ROC AUC: 0.5326, AUCPR: 0.0762
Top 5 features | Precision: 0.0188, Recall: 0.0140, F1: 0.0160, ROC AUC: 0.5065, AUCPR: 0.0170
Top 6 features | Precision: 0.0016, Recall: 0.0012, F1: 0.0014, ROC AUC: 0.5001, AUCPR: 0.0021
Top 7 features | Precision: 0.0115, Recall: 0.0088, F1: 0.0100, ROC AUC: 0.5039, AUCPR: 0.0108
Top 8 features | Precision: 0.0115, Recall: 0.0088, F1: 0.0100, ROC AUC: 0.5039, AUCPR: 0.0108
Top 9 features | Precision: 0.0115, Recall: 0.0088, F1: 0.0100, ROC AUC: 0.5039, AUCPR: 0.0108
Top 10 features | Precision: 0.0115, Recall: 0.0088, F1: 0.0100, ROC AUC: 0.5039, AUCPR: 0.0108


## **Scenario 3: Fraud Ratio Variation**

* Using best split + features
* Vary the percentage of fraud samples used in training: 100%, 90%, ..., 5%


In [ ]:
best_5 = features_sorted[:5]
X_top5 = X_df[best_5]

X_train_full, X_test, y_train_full, y_test = train_test_split(X_top5, y, test_size=0.4, stratify=y, random_state=42)

X_train_fraud = X_train_full[y_train_full == 1]
X_train_normal = X_train_full[y_train_full == 0]

fraud_ratios = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05]
scenario3_results = []

for ratio in fraud_ratios:
    n_fraud = int(len(X_train_fraud) * ratio)
    X_train_combined = pd.concat([X_train_fraud.sample(n=n_fraud, random_state=42), X_train_normal])

    model = IsolationForest(n_estimators=100, max_samples=128, contamination=0.001, random_state=42)
    model.fit(X_train_combined)
    y_pred = model.predict(X_test)
    y_pred = [1 if x == -1 else 0 for x in y_pred]

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred)
    aucpr = auc(recall_vals, precision_vals)

    scenario3_results.append((ratio, precision, recall, f1, roc, aucpr))

for res in scenario3_results:
    print(f"Fraud %: {int(res[0]*100)} | Precision: {res[1]:.4f}, Recall: {res[2]:.4f}, F1: {res[3]:.4f}, ROC AUC: {res[4]:.4f}, AUCPR: {res[5]:.4f}")

## **Scenario 4: Hyperparameter Tuning**

**Tune:**

* n_estimators ∈ {10, 25, 50, 100, 150, 200}
* max_samples ∈ {64, 128, 256, 512, 1024}
* contamination ∈ {0.0004, 0.0008, 0.001, 0.0015, 0.002}


In [ ]:
params_grid = list(product([100, 150, 200], [64, 128, 256], [0.0004, 0.0008, 0.001, 0.0015]))
scenario4_results = []

for n_est, max_samp, contam in params_grid:
    model = IsolationForest(n_estimators=n_est, max_samples=max_samp, contamination=contam, random_state=42)
    model.fit(X_train_combined)
    y_pred = model.predict(X_test)
    y_pred = [1 if x == -1 else 0 for x in y_pred]

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred)
    aucpr = auc(recall_vals, precision_vals)

    scenario4_results.append(((n_est, max_samp, contam), precision, recall, f1, roc, aucpr))

scenario4_results.sort(key=lambda x: x[3], reverse=True)
print("Top 5 configurations by F1-score:")
for res in scenario4_results[:5]:
    print(f"Params: Trees={res[0][0]}, MaxSamples={res[0][1]}, Contam={res[0][2]} | Precision={res[1]:.4f}, Recall={res[2]:.4f}, F1={res[3]:.4f}, ROC={res[4]:.4f}, AUCPR={res[5]:.4f}")

# **Interpretability and Explainability methods**

In [ ]:
lime_exp = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train_combined),
    feature_names=X_df.columns,
    mode='classification',
    class_names=["Normal", "Fraud"]
)

idx = 10
exp = lime_exp.explain_instance(X_test[idx], model.predict, num_features=5)
exp.show_in_notebook()

## **Isolation Forest Built-in Feature Importances**

In [ ]:
importances = model.feature_importances_
plt.barh(X_df.columns, importances)
plt.title("Isolation Forest Feature Importances")
plt.show()

## **Autoencoder Feature Agreement**

In [ ]:
input_dim = X_train_combined.shape[1]
input_layer = Input(shape=(input_dim,))
encoder = Dense(32, activation="relu")(input_layer)
encoder = Dense(16, activation="relu")(encoder)
decoder = Dense(input_dim, activation="linear")(encoder)
autoencoder = Model(input_layer, decoder)
autoencoder.compile(optimizer=Adam(), loss="mse")
autoencoder.fit(X_train_combined, X_train_combined, epochs=10, batch_size=256, verbose=1)

reconstructions = autoencoder.predict(X_test)
mse = np.mean(np.square(X_test - reconstructions), axis=1)

threshold = np.percentile(mse, 95)
anomalies_auto = (mse > threshold).astype(int)

In [ ]:
print(confusion_matrix(y_pred, anomalies_auto))

## **SHAP**

In [ ]:
explainer = shap.Explainer(model.predict, X_test[:100])
shap_values = explainer(X_test[:10])

shap.summary_plot(shap_values, X_test[:10], feature_names=X_df.columns)